# Lecture 4.6 - Basics of cleaning messy text files 
## Part 2 - Grouping blocks of data and extracting information

In this lecture, we will go over a number of cases of messy data, and how to use Python to fix these problems.  This includes

1. Removing unwanted lines.
2. Parsing lines with regular expressions.
3. Working with data blocks spread across multiple lines.

## Reading in current progress

In [1]:
with open('911_Deaths_Grouped.csv') as f:
    content = f.read()
content[:500]

"Gordon M. Aamoth, Jr., 32, Sandler O'Neill + Partners, World Trade Center.\nEdelmiro Abad, 54, Brooklyn, N.Y., Fiduciary Trust Company International, World Trade Center.\nMarie Rose Abad, 49, Keefe, Bruyette&Woods, Inc., World Trade Center.\nAndrew Anthony Abate, 37, Melville, N.Y., Cantor Fitzgerald, World Trade Center.\nVincent Paul Abate, 40, Brooklyn, N.Y., Cantor Fitzgerald, World Trade Center.\nLaurence Christopher Abel, 37, New York City, Cantor Fitzgerald, World Trade Center.\nAlona Abraham, 3"

In [2]:
grouped_lines = content.split('\n')
grouped_lines

["Gordon M. Aamoth, Jr., 32, Sandler O'Neill + Partners, World Trade Center.",
 'Edelmiro Abad, 54, Brooklyn, N.Y., Fiduciary Trust Company International, World Trade Center.',
 'Marie Rose Abad, 49, Keefe, Bruyette&Woods, Inc., World Trade Center.',
 'Andrew Anthony Abate, 37, Melville, N.Y., Cantor Fitzgerald, World Trade Center.',
 'Vincent Paul Abate, 40, Brooklyn, N.Y., Cantor Fitzgerald, World Trade Center.',
 'Laurence Christopher Abel, 37, New York City, Cantor Fitzgerald, World Trade Center.',
 'Alona Abraham, 30, Ashdod, Israel, Passenger, United 175, World Trade Center.',
 'William F. Abrahamson, 55, Westchester County, N.Y., Marsh&McLennan Companies, Inc., World Trade Center.',
 'Richard Anthony Aceto, 42, Marsh&McLennan Companies, Inc., World Trade Center.',
 'Heinrich Bernhard Ackermann, 38, Aon Corporation, World Trade Center.',
 'Paul Acquaviva, 29, Glen Rock, N.J., Cantor Fitzgerald, World Trade Center.',
 'Christian Adams, 37, Passenger, United 93, Shanksville, Pa.',


## Preprocessing 

Below I have transfered over the preprocessing functions and applied them to the data.

In [3]:
# Imports
from composable import pipeable
from composable.strict import map

In [4]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))

In [5]:
(grouped_lines
>> map(add_missing_period)
>> map(fix_world_trade)
)

["Gordon M. Aamoth, Jr., 32, Sandler O'Neill + Partners, World Trade Center.",
 'Edelmiro Abad, 54, Brooklyn, N.Y., Fiduciary Trust Company International, World Trade Center.',
 'Marie Rose Abad, 49, Keefe, Bruyette&Woods, Inc., World Trade Center.',
 'Andrew Anthony Abate, 37, Melville, N.Y., Cantor Fitzgerald, World Trade Center.',
 'Vincent Paul Abate, 40, Brooklyn, N.Y., Cantor Fitzgerald, World Trade Center.',
 'Laurence Christopher Abel, 37, New York City, Cantor Fitzgerald, World Trade Center.',
 'Alona Abraham, 30, Ashdod, Israel, Passenger, United 175, World Trade Center.',
 'William F. Abrahamson, 55, Westchester County, N.Y., Marsh&McLennan Companies, Inc., World Trade Center.',
 'Richard Anthony Aceto, 42, Marsh&McLennan Companies, Inc., World Trade Center.',
 'Heinrich Bernhard Ackermann, 38, Aon Corporation, World Trade Center.',
 'Paul Acquaviva, 29, Glen Rock, N.J., Cantor Fitzgerald, World Trade Center.',
 'Christian Adams, 37, Passenger, United 93, Shanksville, Pa.',


In [6]:
# For convenience I will give these a name
prepped_lines = (grouped_lines 
                >> map(add_missing_period)
                >> map(fix_world_trade)
                )
prepped_lines

["Gordon M. Aamoth, Jr., 32, Sandler O'Neill + Partners, World Trade Center.",
 'Edelmiro Abad, 54, Brooklyn, N.Y., Fiduciary Trust Company International, World Trade Center.',
 'Marie Rose Abad, 49, Keefe, Bruyette&Woods, Inc., World Trade Center.',
 'Andrew Anthony Abate, 37, Melville, N.Y., Cantor Fitzgerald, World Trade Center.',
 'Vincent Paul Abate, 40, Brooklyn, N.Y., Cantor Fitzgerald, World Trade Center.',
 'Laurence Christopher Abel, 37, New York City, Cantor Fitzgerald, World Trade Center.',
 'Alona Abraham, 30, Ashdod, Israel, Passenger, United 175, World Trade Center.',
 'William F. Abrahamson, 55, Westchester County, N.Y., Marsh&McLennan Companies, Inc., World Trade Center.',
 'Richard Anthony Aceto, 42, Marsh&McLennan Companies, Inc., World Trade Center.',
 'Heinrich Bernhard Ackermann, 38, Aon Corporation, World Trade Center.',
 'Paul Acquaviva, 29, Glen Rock, N.J., Cantor Fitzgerald, World Trade Center.',
 'Christian Adams, 37, Passenger, United 93, Shanksville, Pa.',


## Regular expression from lab 2

Below I have attempted to combine all of the regular expressions from lab 2

In [33]:
import re
line_parts = re.compile('^(.+), (\?\?|\d{1,3}),(.*?)( Passenger,| Flight Crew,)?( United \d{2,3},| American \d{2,3},)?( World Trade Center| Pentagon| Shanksville, Pa)(, died \d{1,2}/\d{1,2}/\d{1,2})?\.$')

In [34]:
prepped_lines[2402]

'Jesus Sanchez, 45, Flight Crew, United 175, World Trade Center.'

In [35]:
line_parts.search(prepped_lines[2402]).groups()

('Jesus Sanchez',
 '45',
 '',
 ' Flight Crew,',
 ' United 175,',
 ' World Trade Center',
 None)

#### Always check for non-matches

In [10]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

## Verbose regular expressions

**Pros:**
* Spread over multiple lines
* Allow comments

**Cons:**
* Ignore white space outside `()`
* Require escaping spaces `\ `

In [11]:
# Without Using VERBOSE 
regex_email = re.compile(r'^([a-z0-9_\.-]+)@([0-9a-z\.-]+)\.([a-z\.]{2, 6})$')

In [12]:
# Using VERBOSE 
regex_email = re.compile(r""" 
                        ^([a-z0-9_\.-]+)			 # local Part 
                        @							 # single @ sign 
                        ([0-9a-z\.-]+)			 	 # Domain name 
                        \.						 	 # single Dot . 
                        ([a-z]{2,6})$				 # Top level Domain 
                        """,re.VERBOSE)

## Another example.

This example, from the Python docs, shows how to space out an OR section across multiple lines.

In [13]:
charref = re.compile(r"""
 &[#]                # Start of a numeric entity reference
 (
     0[0-7]+         # Octal form
   | [0-9]+          # Decimal form
   | x[0-9a-fA-F]+   # Hexadecimal form
 )
 ;                   # Trailing semicolon
""", re.VERBOSE)

## Cleaning up our regular expr

<h2> <font color="red"> Exercise 4.6.1 - Clean up the regular expression </font> </h2>

To clean up the regular expression, 

1. Replace all spaces with `\ ` or `\s` (I prefer the second)
2. Turn the string into a multi-line string.
3. Spread the parts over many lines
4. Add comments.

> Describe the bug here

In [42]:
line_parts = re.compile(r"""
^(.+),\s(\?\?|\d{1,3}),
(.*?)(\sPassenger, | \sFlight Crew,)
?(\s United\s\d{2,3},|\s American\s\d{2,3},)
?(\sWorld\sTrade\sCenter|\sPentagon|\sShanksville,\sPa)
(,\sdied\s\d{1,2}/\d{1,2}/\d{1,2})?\.$""",re.VERBOSE)

In [43]:
line_parts.search(prepped_lines[2402]).groups()

('Jesus Sanchez',
 '45',
 ' Flight Crew,',
 None,
 ' United 175,',
 ' World Trade Center',
 None)

## Progress so far

In [44]:
# Imports
from composable import pipeable
from composable.strict import map

In [45]:
# Reg Ex for a line
line_parts = re.compile(r'''^(.+),
(
      \s\?\?                          # ??
    | \s\d{1,3}                       # or age
),
(.*?)                                 # Includes hometown and 
(
        \sPassenger,                  # Optional flight status
    |   \sFlightsCrew,
)?
(
      \sUnited\s\d{2,3},              # Optional flight
    | \sAmericans\d{2,3},
)?
(
       \sWorld\sTrade\sCenter         # Location
    |  \sPentagon
    |  \sShanksville,\sPa
)
(
    ,\sdied\s\d{1,2}/\d{1,2}/\d{1,2}  # Optional date of death
)?
\.$''', re.VERBOSE)

In [46]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))
# New
get_line_parts = pipeable(lambda line: line_parts.search(line).groups(default=''))

In [47]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

In [48]:
split_lines =  (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(get_line_parts)
                )
split_lines

[('Gordon M. Aamoth, Jr.',
  ' 32',
  " Sandler O'Neill + Partners,",
  '',
  '',
  ' World Trade Center',
  ''),
 ('Edelmiro Abad',
  ' 54',
  ' Brooklyn, N.Y., Fiduciary Trust Company International,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Marie Rose Abad',
  ' 49',
  ' Keefe, Bruyette&Woods, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Andrew Anthony Abate',
  ' 37',
  ' Melville, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Vincent Paul Abate',
  ' 40',
  ' Brooklyn, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Laurence Christopher Abel',
  ' 37',
  ' New York City, Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Alona Abraham',
  ' 30',
  ' Ashdod, Israel,',
  ' Passenger,',
  ' United 175,',
  ' World Trade Center',
  ''),
 ('William F. Abrahamson',
  ' 55',
  ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Richard Anthony Ac

## Pulling out and cleaning up names

Sometimes it is useful to pull the various columns apart and clean them up separately.  To illustrate, will will pull out and clean up the names. We can do this using the `get` function from `toolz.curried` which *gets* the value from a list at a given index.

In [49]:
from toolz.curried import get

In [50]:
(split_lines
>> map(get(0))
)

['Gordon M. Aamoth, Jr.',
 'Edelmiro Abad',
 'Marie Rose Abad',
 'Andrew Anthony Abate',
 'Vincent Paul Abate',
 'Laurence Christopher Abel',
 'Alona Abraham',
 'William F. Abrahamson',
 'Richard Anthony Aceto',
 'Heinrich Bernhard Ackermann',
 'Paul Acquaviva',
 'Christian Adams',
 'Donald LaRoy Adams',
 'Patrick Adams',
 'Shannon Lewis Adams',
 'Stephen George Adams',
 'Ignatius Udo Adanga',
 'Christy A. Addamo',
 'Terence Edward Adderley, Jr.',
 'Sophia B. Addo',
 'Lee Adler',
 'Daniel Thomas Afflitto',
 'Emmanuel Akwasi Afuakwah',
 'Alok Agarwal',
 'Mukul Kumar Agarwala',
 'Joseph Agnello',
 'David Scott Agnes',
 'Joao Alberto da Fonseca Aguiar, Jr.',
 'Brian G. Ahearn',
 'Jeremiah Joseph Ahern',
 'Joanne Marie Ahladiotis',
 'Shabbir Ahmed',
 'Terrance Andre Aiken',
 'Godwin O. Ajala',
 'Trudi M. Alagero',
 'Andrew Alameno',
 'Margaret Ann Alario',
 'Gary M. Albero',
 'Jon Leslie Albert',
 'Peter Craig Alderman',
 'Jacquelyn Delaine Aldridge-Frederick',
 'David D. Alger',
 'Ernest 

Now we can clean up these name by removing commas.

In [117]:
remove_commas = lambda s: s.replace(',', '')

(split_lines
>> map(get(0))
>> map(remove_commas)
)

['Gordon M. Aamoth Jr.',
 'Edelmiro Abad',
 'Marie Rose Abad',
 'Andrew Anthony Abate',
 'Vincent Paul Abate',
 'Laurence Christopher Abel',
 'Alona Abraham',
 'William F. Abrahamson',
 'Richard Anthony Aceto',
 'Heinrich Bernhard Ackermann',
 'Paul Acquaviva',
 'Christian Adams',
 'Donald LaRoy Adams',
 'Patrick Adams',
 'Shannon Lewis Adams',
 'Stephen George Adams',
 'Ignatius Udo Adanga',
 'Christy A. Addamo',
 'Terence Edward Adderley Jr.',
 'Sophia B. Addo',
 'Lee Adler',
 'Daniel Thomas Afflitto',
 'Emmanuel Akwasi Afuakwah',
 'Alok Agarwal',
 'Mukul Kumar Agarwala',
 'Joseph Agnello',
 'David Scott Agnes',
 'Joao Alberto da Fonseca Aguiar Jr.',
 'Brian G. Ahearn',
 'Jeremiah Joseph Ahern',
 'Joanne Marie Ahladiotis',
 'Shabbir Ahmed',
 'Terrance Andre Aiken',
 'Godwin O. Ajala',
 'Trudi M. Alagero',
 'Andrew Alameno',
 'Margaret Ann Alario',
 'Gary M. Albero',
 'Jon Leslie Albert',
 'Peter Craig Alderman',
 'Jacquelyn Delaine Aldridge-Frederick',
 'David D. Alger',
 'Ernest Ali

## Pulling out and cleaning up ages

NExt, we will pull out and clean the ages.  In this case, we should replace the missing values, currently `'??'`, to blanks.

In [52]:
remove_quest_mark = lambda s: s.replace('??', '')

(split_lines
>> map(get(1))
>> map(remove_quest_mark)
)

[' 32',
 ' 54',
 ' 49',
 ' 37',
 ' 40',
 ' 37',
 ' 30',
 ' 55',
 ' 42',
 ' 38',
 ' 29',
 ' 37',
 ' 28',
 ' 61',
 ' 25',
 ' 51',
 ' 62',
 ' 28',
 ' 22',
 ' 36',
 ' 48',
 ' 32',
 ' 37',
 ' 36',
 ' 37',
 ' 35',
 ' 46',
 ' 30',
 ' 43',
 ' 74',
 ' 27',
 ' 47',
 ' 30',
 ' 33',
 ' 37',
 ' 37',
 ' 41',
 ' 39',
 ' 46',
 ' 25',
 ' 46',
 ' 57',
 ' 43',
 ' 51',
 ' 44',
 ' 39',
 ' 31',
 ' 30',
 ' 36',
 ' 48',
 ' 41',
 ' 31',
 ' 23',
 ' 38',
 ' 25',
 ' 60',
 ' 40',
 ' 60',
 ' 43',
 ' 41',
 ' 32',
 ' 29',
 ' 28',
 ' 42',
 ' 35',
 ' 26',
 ' 57',
 ' 53',
 ' 52',
 ' 34',
 ' 43',
 ' 37',
 ' 63',
 ' 38',
 ' 54',
 ' 52',
 ' 23',
 ' 44',
 ' 32',
 ' 48',
 ' 26',
 ' 55',
 ' 26',
 ' 26',
 ' 36',
 ' 45',
 ' 32',
 ' 38',
 ' 37',
 ' 34',
 ' 52',
 ' 29',
 ' 48',
 ' 50',
 ' 49',
 ' 37',
 ' 47',
 ' 53',
 ' 25',
 ' 21',
 ' 35',
 ' 44',
 ' 38',
 ' 38',
 ' 58',
 ' 41',
 ' 36',
 ' 47',
 ' 48',
 ' 35',
 ' 43',
 ' 44',
 ' 29',
 ' 28',
 ' 53',
 ' 43',
 ' 36',
 ' 43',
 ' 33',
 ' 44',
 ' 35',
 ' 42',
 ' 53',
 ' 35',
 ' 53',


## Progress so far

In [53]:
# Imports
from composable import pipeable
from composable.strict import map

In [54]:
# Reg Ex for a line
line_parts = re.compile(r'''^(.+),
(
      \s\?\?                          # ??
    | \s\d{1,3}                       # or age
),
(.*?)                                 # Includes hometown and 
(
        \sPassenger,                  # Optional flight status
    |   \sFlightsCrew,
)?
(
      \sUnited\s\d{2,3},              # Optional flight
    | \sAmericans\d{2,3},
)?
(
       \sWorld\sTrade\sCenter         # Location
    |  \sPentagon
    |  \sShanksville,\sPa
)
(
    ,\sdied\s\d{1,2}/\d{1,2}/\d{1,2}  # Optional date of death
)?
\.$''', re.VERBOSE)

In [55]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))
get_line_parts = pipeable(lambda line: line_parts.search(line).groups(default=''))
# New
remove_commas = lambda s: s.replace(',', '')
remove_quest_mark = lambda s: s.replace('??', '')

In [56]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

In [57]:
split_lines =  (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(get_line_parts)
                )
split_lines

[('Gordon M. Aamoth, Jr.',
  ' 32',
  " Sandler O'Neill + Partners,",
  '',
  '',
  ' World Trade Center',
  ''),
 ('Edelmiro Abad',
  ' 54',
  ' Brooklyn, N.Y., Fiduciary Trust Company International,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Marie Rose Abad',
  ' 49',
  ' Keefe, Bruyette&Woods, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Andrew Anthony Abate',
  ' 37',
  ' Melville, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Vincent Paul Abate',
  ' 40',
  ' Brooklyn, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Laurence Christopher Abel',
  ' 37',
  ' New York City, Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Alona Abraham',
  ' 30',
  ' Ashdod, Israel,',
  ' Passenger,',
  ' United 175,',
  ' World Trade Center',
  ''),
 ('William F. Abrahamson',
  ' 55',
  ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Richard Anthony Ac

In [58]:
names =  (split_lines
        >> map(get(0))
        >> map(remove_commas)
        )
names

['Gordon M. Aamoth Jr.',
 'Edelmiro Abad',
 'Marie Rose Abad',
 'Andrew Anthony Abate',
 'Vincent Paul Abate',
 'Laurence Christopher Abel',
 'Alona Abraham',
 'William F. Abrahamson',
 'Richard Anthony Aceto',
 'Heinrich Bernhard Ackermann',
 'Paul Acquaviva',
 'Christian Adams',
 'Donald LaRoy Adams',
 'Patrick Adams',
 'Shannon Lewis Adams',
 'Stephen George Adams',
 'Ignatius Udo Adanga',
 'Christy A. Addamo',
 'Terence Edward Adderley Jr.',
 'Sophia B. Addo',
 'Lee Adler',
 'Daniel Thomas Afflitto',
 'Emmanuel Akwasi Afuakwah',
 'Alok Agarwal',
 'Mukul Kumar Agarwala',
 'Joseph Agnello',
 'David Scott Agnes',
 'Joao Alberto da Fonseca Aguiar Jr.',
 'Brian G. Ahearn',
 'Jeremiah Joseph Ahern',
 'Joanne Marie Ahladiotis',
 'Shabbir Ahmed',
 'Terrance Andre Aiken',
 'Godwin O. Ajala',
 'Trudi M. Alagero',
 'Andrew Alameno',
 'Margaret Ann Alario',
 'Gary M. Albero',
 'Jon Leslie Albert',
 'Peter Craig Alderman',
 'Jacquelyn Delaine Aldridge-Frederick',
 'David D. Alger',
 'Ernest Ali

In [59]:
ages =  (split_lines
        >> map(get(1))
        >> map(remove_quest_mark)
        )
ages

[' 32',
 ' 54',
 ' 49',
 ' 37',
 ' 40',
 ' 37',
 ' 30',
 ' 55',
 ' 42',
 ' 38',
 ' 29',
 ' 37',
 ' 28',
 ' 61',
 ' 25',
 ' 51',
 ' 62',
 ' 28',
 ' 22',
 ' 36',
 ' 48',
 ' 32',
 ' 37',
 ' 36',
 ' 37',
 ' 35',
 ' 46',
 ' 30',
 ' 43',
 ' 74',
 ' 27',
 ' 47',
 ' 30',
 ' 33',
 ' 37',
 ' 37',
 ' 41',
 ' 39',
 ' 46',
 ' 25',
 ' 46',
 ' 57',
 ' 43',
 ' 51',
 ' 44',
 ' 39',
 ' 31',
 ' 30',
 ' 36',
 ' 48',
 ' 41',
 ' 31',
 ' 23',
 ' 38',
 ' 25',
 ' 60',
 ' 40',
 ' 60',
 ' 43',
 ' 41',
 ' 32',
 ' 29',
 ' 28',
 ' 42',
 ' 35',
 ' 26',
 ' 57',
 ' 53',
 ' 52',
 ' 34',
 ' 43',
 ' 37',
 ' 63',
 ' 38',
 ' 54',
 ' 52',
 ' 23',
 ' 44',
 ' 32',
 ' 48',
 ' 26',
 ' 55',
 ' 26',
 ' 26',
 ' 36',
 ' 45',
 ' 32',
 ' 38',
 ' 37',
 ' 34',
 ' 52',
 ' 29',
 ' 48',
 ' 50',
 ' 49',
 ' 37',
 ' 47',
 ' 53',
 ' 25',
 ' 21',
 ' 35',
 ' 44',
 ' 38',
 ' 38',
 ' 58',
 ' 41',
 ' 36',
 ' 47',
 ' 48',
 ' 35',
 ' 43',
 ' 44',
 ' 29',
 ' 28',
 ' 53',
 ' 43',
 ' 36',
 ' 43',
 ' 33',
 ' 44',
 ' 35',
 ' 42',
 ' 53',
 ' 35',
 ' 53',


<h2> <font color="red"> Exercise 4.6.2 - Separating and cleaning other columns. </font> </h2>

To clean up the following columns 

1. Grab the date of death and replace the missing values with `9/11/2001`
2. Grab the locations (e.g. `World Trade Center`) and remove the comma from `'Shanksville, Pa.`
3. Grab the flights.
4. Grab the passenger status.

**Note:** Be sure to strip whitespace from all of them.

In [62]:
fix_death = pipeable(lambda line: line.replace('', '9/11/2001'))

In [64]:
death =  (split_lines
        >> map(get(6))
        >> map(fix_death)
        )
death

['9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001,9/11/2001 9/11/2001d9/11/2001i9/11/2001e9/11/2001d9/11/2001 9/11/200199/11/2001/9/11/200119/11/200159/11/2001/9/11/200109/11/200119/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11

In [70]:
fix_pa = pipeable(lambda line: line.replace('Shanksville, Pa', 'Shanksville Pa.'))

In [71]:
location =  (split_lines
        >> map(get(5))
        >> map(fix_pa)
        )
location

[' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' Shanksville Pa.',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Center',
 ' World Trade Cent

In [94]:
fix_flight = pipeable(lambda line: line.replace(',, ''))

SyntaxError: EOL while scanning string literal (<ipython-input-94-eab0448561cc>, line 1)

In [95]:
flight =  (split_lines
        >> map(get(4))
        >> map(fix_flight)
        )
flight

['',
 '',
 '',
 '',
 '',
 '',
 ' United 175,',
 '',
 '',
 '',
 '',
 ' United 93,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' United 11,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' United 11,',
 ' United 11,',
 '',
 '',
 '',
 ' United 11,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' United 11,',
 '',
 '',
 '',
 '',
 '',
 ' United 11,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' United 175,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' United 11,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' United 175,',
 '',
 ' United 93,',
 '',
 ' United 93,',
 '',
 '',
 ' United 93,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ''

In [84]:
passenger =  (split_lines
        >> map(get(3))
        )
passenger

['',
 '',
 '',
 '',
 '',
 '',
 ' Passenger,',
 '',
 '',
 '',
 '',
 ' Passenger,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' Passenger,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' Passenger,',
 ' Passenger,',
 '',
 '',
 '',
 ' Passenger,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' Passenger,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' Passenger,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' Passenger,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' Passenger,',
 '',
 '',
 '',
 ' Passenger,',
 '',
 '',
 ' Passenger,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ''

## Grabbing the troubling bit

We have made significant progress, but still need to work on the third entry, which contains the hometown and employment information.  Again, we can do this using the `get` function from `toolz.curried` which *gets* the value from a list at a given index.

In [73]:
troubling_bit = (split_lines
                >> map(get(2))
                )
troubling_bit

[" Sandler O'Neill + Partners,",
 ' Brooklyn, N.Y., Fiduciary Trust Company International,',
 ' Keefe, Bruyette&Woods, Inc.,',
 ' Melville, N.Y., Cantor Fitzgerald,',
 ' Brooklyn, N.Y., Cantor Fitzgerald,',
 ' New York City, Cantor Fitzgerald,',
 ' Ashdod, Israel,',
 ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
 ' Marsh&McLennan Companies, Inc.,',
 ' Aon Corporation,',
 ' Glen Rock, N.J., Cantor Fitzgerald,',
 '',
 ' Cantor Fitzgerald,',
 ' Fuji Bank, Ltd. security,',
 ' Cantor Fitzgerald,',
 ' New York City, Windows on the World,',
 ' Bronx, N.Y., New York Metropolitan Transportation Council,',
 ' New Hyde Park, N.Y., Marsh&McLennan Companies, Inc.,',
 ' New York City, Fred Alger Management, Inc.,',
 ' Bronx, N.Y., Windows on the World,',
 ' Cantor Fitzgerald,',
 ' Manalapan, N.J., Cantor Fitzgerald,',
 ' Windows on the World,',
 ' Cantor Fitzgerald,',
 ' Fiduciary Trust Company International,',
 ' Belle Harbor, N.Y., New York City Fire Department,',
 ' Port Washingto

## Progressively filtering out states

We will start by matching two of the most common states, NY and NJ.

In [74]:
state = re.compile(', (N\.Y\.|N\.J\.),?')
# Rows that match
[(l, state.search(l)) for l in troubling_bit]

[(" Sandler O'Neill + Partners,", None),
 (' Brooklyn, N.Y., Fiduciary Trust Company International,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' Keefe, Bruyette&Woods, Inc.,', None),
 (' Melville, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' Brooklyn, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' New York City, Cantor Fitzgerald,', None),
 (' Ashdod, Israel,', None),
 (' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
  <re.Match object; span=(19, 26), match=', N.Y.,'>),
 (' Marsh&McLennan Companies, Inc.,', None),
 (' Aon Corporation,', None),
 (' Glen Rock, N.J., Cantor Fitzgerald,',
  <re.Match object; span=(10, 17), match=', N.J.,'>),
 ('', None),
 (' Cantor Fitzgerald,', None),
 (' Fuji Bank, Ltd. security,', None),
 (' Cantor Fitzgerald,', None),
 (' New York City, Windows on the World,', None),
 (' Bronx, N.Y., New York Metropolitan Transportation Council,',
  <re.Match object;

and inspecting all rows that don't match for additional states or problems

In [75]:
[(i, l) for i, l in enumerate(troubling_bit) if not state.search(l)]

[(0, " Sandler O'Neill + Partners,"),
 (2, ' Keefe, Bruyette&Woods, Inc.,'),
 (5, ' New York City, Cantor Fitzgerald,'),
 (6, ' Ashdod, Israel,'),
 (8, ' Marsh&McLennan Companies, Inc.,'),
 (9, ' Aon Corporation,'),
 (11, ''),
 (12, ' Cantor Fitzgerald,'),
 (13, ' Fuji Bank, Ltd. security,'),
 (14, ' Cantor Fitzgerald,'),
 (15, ' New York City, Windows on the World,'),
 (18, ' New York City, Fred Alger Management, Inc.,'),
 (20, ' Cantor Fitzgerald,'),
 (22, ' Windows on the World,'),
 (23, ' Cantor Fitzgerald,'),
 (24, ' Fiduciary Trust Company International,'),
 (29, ' New Jersey, New York State Department of Taxation and Finance,'),
 (32, ' Marsh&McLennan consultant,'),
 (33, ' Summit Security Services, Inc.,'),
 (34, ' New York City, Marsh&McLennan Companies, Inc.,'),
 (39,
  ' New York City, Risk Waters Group conference attendee from Bloomberg L.P.,'),
 (41, ' New York City, Fred Alger Management, Inc.,'),
 (43, ' Cantor Fitzgerald,'),
 (45, ' Cantor Fitzgerald,'),
 (49, ' Stoneha

## Fixing a common problem.

Notice that many rows simply contain ` New York City,` without the state.  Let's fix this problem in our preprocessing step.

In [76]:
grouped_lines[41]

'David D. Alger, 57, New York City, Fred Alger Management, Inc., World Trade Center.'

In [77]:
fix_nyc = pipeable(lambda line: line.replace(', New York City,', ', New York City, N.Y.,'))
grouped_lines[41] >> fix_nyc

'David D. Alger, 57, New York City, N.Y., Fred Alger Management, Inc., World Trade Center.'

## Progress so far

In [78]:
# Imports
from composable import pipeable
from composable.strict import map

In [79]:
# Reg Ex for a line
line_parts = re.compile(r'''^(.+),
(
      \s\?\?                          # ??
    | \s\d{1,3}                       # or age
),
(.*?)                                 # Includes hometown and 
(
        \sPassenger,                  # Optional flight status
    |   \sFlightsCrew,
)?
(
      \sUnited\s\d{2,3},              # Optional flight
    | \sAmericans\d{2,3},
)?
(
       \sWorld\sTrade\sCenter         # Location
    |  \sPentagon
    |  \sShanksville,\sPa
)
(
    ,\sdied\s\d{1,2}/\d{1,2}/\d{1,2}  # Optional date of death
)?
\.$''', re.VERBOSE)

In [96]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))
get_line_parts = pipeable(lambda line: line_parts.search(line).groups(default=''))
remove_commas = lambda s: s.replace(',', '')
# New
fix_nyc = pipeable(lambda line: line.replace(', New York City,', ', New York City, N.Y.,'))

In [97]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

In [98]:
split_lines =  (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(fix_nyc)
                >> map(get_line_parts)
                )
split_lines

[('Gordon M. Aamoth, Jr.',
  ' 32',
  " Sandler O'Neill + Partners,",
  '',
  '',
  ' World Trade Center',
  ''),
 ('Edelmiro Abad',
  ' 54',
  ' Brooklyn, N.Y., Fiduciary Trust Company International,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Marie Rose Abad',
  ' 49',
  ' Keefe, Bruyette&Woods, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Andrew Anthony Abate',
  ' 37',
  ' Melville, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Vincent Paul Abate',
  ' 40',
  ' Brooklyn, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Laurence Christopher Abel',
  ' 37',
  ' New York City, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Alona Abraham',
  ' 30',
  ' Ashdod, Israel,',
  ' Passenger,',
  ' United 175,',
  ' World Trade Center',
  ''),
 ('William F. Abrahamson',
  ' 55',
  ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Richard Anth

In [99]:
names =  (split_lines
        >> map(get(0))
        >> map(remove_commas)
        )
names

['Gordon M. Aamoth Jr.',
 'Edelmiro Abad',
 'Marie Rose Abad',
 'Andrew Anthony Abate',
 'Vincent Paul Abate',
 'Laurence Christopher Abel',
 'Alona Abraham',
 'William F. Abrahamson',
 'Richard Anthony Aceto',
 'Heinrich Bernhard Ackermann',
 'Paul Acquaviva',
 'Christian Adams',
 'Donald LaRoy Adams',
 'Patrick Adams',
 'Shannon Lewis Adams',
 'Stephen George Adams',
 'Ignatius Udo Adanga',
 'Christy A. Addamo',
 'Terence Edward Adderley Jr.',
 'Sophia B. Addo',
 'Lee Adler',
 'Daniel Thomas Afflitto',
 'Emmanuel Akwasi Afuakwah',
 'Alok Agarwal',
 'Mukul Kumar Agarwala',
 'Joseph Agnello',
 'David Scott Agnes',
 'Joao Alberto da Fonseca Aguiar Jr.',
 'Brian G. Ahearn',
 'Jeremiah Joseph Ahern',
 'Joanne Marie Ahladiotis',
 'Shabbir Ahmed',
 'Terrance Andre Aiken',
 'Godwin O. Ajala',
 'Trudi M. Alagero',
 'Andrew Alameno',
 'Margaret Ann Alario',
 'Gary M. Albero',
 'Jon Leslie Albert',
 'Peter Craig Alderman',
 'Jacquelyn Delaine Aldridge-Frederick',
 'David D. Alger',
 'Ernest Ali

In [100]:
troubling_bit = (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(fix_nyc)
                >> map(get_line_parts)
                >> map(get(2))
                )
troubling_bit

[" Sandler O'Neill + Partners,",
 ' Brooklyn, N.Y., Fiduciary Trust Company International,',
 ' Keefe, Bruyette&Woods, Inc.,',
 ' Melville, N.Y., Cantor Fitzgerald,',
 ' Brooklyn, N.Y., Cantor Fitzgerald,',
 ' New York City, N.Y., Cantor Fitzgerald,',
 ' Ashdod, Israel,',
 ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
 ' Marsh&McLennan Companies, Inc.,',
 ' Aon Corporation,',
 ' Glen Rock, N.J., Cantor Fitzgerald,',
 '',
 ' Cantor Fitzgerald,',
 ' Fuji Bank, Ltd. security,',
 ' Cantor Fitzgerald,',
 ' New York City, N.Y., Windows on the World,',
 ' Bronx, N.Y., New York Metropolitan Transportation Council,',
 ' New Hyde Park, N.Y., Marsh&McLennan Companies, Inc.,',
 ' New York City, N.Y., Fred Alger Management, Inc.,',
 ' Bronx, N.Y., Windows on the World,',
 ' Cantor Fitzgerald,',
 ' Manalapan, N.J., Cantor Fitzgerald,',
 ' Windows on the World,',
 ' Cantor Fitzgerald,',
 ' Fiduciary Trust Company International,',
 ' Belle Harbor, N.Y., New York City Fire Department,',

## Adding more states

Next, we will start adding start to our pattern, and again looking for additional states/problems.  For example, let's add the `Mass.` and `D.C.` patterns.

In [101]:
state = re.compile(', (N\.Y\.|N\.J\.|Mass\.|D\.C\.),?')
[(l, state.search(l)) for l in troubling_bit if state.search(l)]

[(' Brooklyn, N.Y., Fiduciary Trust Company International,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' Melville, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' Brooklyn, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' New York City, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(14, 21), match=', N.Y.,'>),
 (' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
  <re.Match object; span=(19, 26), match=', N.Y.,'>),
 (' Glen Rock, N.J., Cantor Fitzgerald,',
  <re.Match object; span=(10, 17), match=', N.J.,'>),
 (' New York City, N.Y., Windows on the World,',
  <re.Match object; span=(14, 21), match=', N.Y.,'>),
 (' Bronx, N.Y., New York Metropolitan Transportation Council,',
  <re.Match object; span=(6, 13), match=', N.Y.,'>),
 (' New Hyde Park, N.Y., Marsh&McLennan Companies, Inc.,',
  <re.Match object; span=(14, 21), match=', N.Y.,'>),
 (' New York City, N.Y., Fred Alger Management, In

In [102]:
[(i, l) for i, l in enumerate(troubling_bit) if not state.search(l)]

[(0, " Sandler O'Neill + Partners,"),
 (2, ' Keefe, Bruyette&Woods, Inc.,'),
 (6, ' Ashdod, Israel,'),
 (8, ' Marsh&McLennan Companies, Inc.,'),
 (9, ' Aon Corporation,'),
 (11, ''),
 (12, ' Cantor Fitzgerald,'),
 (13, ' Fuji Bank, Ltd. security,'),
 (14, ' Cantor Fitzgerald,'),
 (20, ' Cantor Fitzgerald,'),
 (22, ' Windows on the World,'),
 (23, ' Cantor Fitzgerald,'),
 (24, ' Fiduciary Trust Company International,'),
 (29, ' New Jersey, New York State Department of Taxation and Finance,'),
 (32, ' Marsh&McLennan consultant,'),
 (33, ' Summit Security Services, Inc.,'),
 (43, ' Cantor Fitzgerald,'),
 (45, ' Cantor Fitzgerald,'),
 (51, ' Cantor Fitzgerald, Forte Food Service,'),
 (52, ' Windows on the World,'),
 (54, ' Windows on the World,'),
 (55, ' Marsh&McLennan Companies, Inc.,'),
 (56, ' Fiduciary Trust Company International,'),
 (57, ' ABM Industries Inc.,'),
 (58, ' New York City Fire Department,'),
 (59, ' Port Authority of New York and New Jersey first responders,'),
 (61, ' 

<h2> <font color="red"> Exercise 4.6.2 - Continue the process. </font> </h2>

Now it is your turn.  You should

1. Keep adding states to the pattern.
2. Add preprocessing steps to fix any issues.

In [103]:
state = re.compile(', (N\.Y\.|N\.J\.|Mass\.|D\.C\.|Iowa|Maine|Utah|Texas),?')
[(l, state.search(l)) for l in troubling_bit if state.search(l)]

[(' Brooklyn, N.Y., Fiduciary Trust Company International,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' Melville, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' Brooklyn, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' New York City, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(14, 21), match=', N.Y.,'>),
 (' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
  <re.Match object; span=(19, 26), match=', N.Y.,'>),
 (' Glen Rock, N.J., Cantor Fitzgerald,',
  <re.Match object; span=(10, 17), match=', N.J.,'>),
 (' New York City, N.Y., Windows on the World,',
  <re.Match object; span=(14, 21), match=', N.Y.,'>),
 (' Bronx, N.Y., New York Metropolitan Transportation Council,',
  <re.Match object; span=(6, 13), match=', N.Y.,'>),
 (' New Hyde Park, N.Y., Marsh&McLennan Companies, Inc.,',
  <re.Match object; span=(14, 21), match=', N.Y.,'>),
 (' New York City, N.Y., Fred Alger Management, In

<h2> <font color="red"> Exercise 4.6.3 - Make your solution verbose </font> </h2>

Now make your solution to the last problem verbose.  Also reorder the cases so that similar cases are close and add comments.  Finally, change the regular expression to capture the parts before and after the state.

In [107]:
state = re.compile('''
^(.*?)
,?\s                    # Optional comman
(
       N\.Y\.           
    |  N\.J\.
    |  D\.C\.
    |  Texas
    |  Iowa
    |  Maine
    |  Utah
    |  Mass\.

)
,
(.*?)$
''', re.VERBOSE)

## Splitting the troubling bit

Now that we have a way to identify rows that have home addresses (through matching the state), we will split up this data.  We will do this by considering three cases.

1. Blank entry become three blanks (for town, state, employer).
2. Lines that match the states regex will get split by this pattern.
3. The remaining lines hold only the employer and become `'','',entry`

In [115]:
def split_troubling_bit(entry):
    if len(entry) == 0:
        return ('', '', '')
    elif state.search(entry):
        return state.search(entry).groups(default='')
    else:
        return ('', '', entry)

In [116]:
( troubling_bit
 >> map(split_troubling_bit)
)

[('', '', " Sandler O'Neill + Partners,"),
 (' Brooklyn', 'N.Y.', ' Fiduciary Trust Company International,'),
 ('', '', ' Keefe, Bruyette&Woods, Inc.,'),
 (' Melville', 'N.Y.', ' Cantor Fitzgerald,'),
 (' Brooklyn', 'N.Y.', ' Cantor Fitzgerald,'),
 (' New York City', 'N.Y.', ' Cantor Fitzgerald,'),
 ('', '', ' Ashdod, Israel,'),
 (' Westchester County', 'N.Y.', ' Marsh&McLennan Companies, Inc.,'),
 ('', '', ' Marsh&McLennan Companies, Inc.,'),
 ('', '', ' Aon Corporation,'),
 (' Glen Rock', 'N.J.', ' Cantor Fitzgerald,'),
 ('', '', ''),
 ('', '', ' Cantor Fitzgerald,'),
 ('', '', ' Fuji Bank, Ltd. security,'),
 ('', '', ' Cantor Fitzgerald,'),
 (' New York City', 'N.Y.', ' Windows on the World,'),
 (' Bronx', 'N.Y.', ' New York Metropolitan Transportation Council,'),
 (' New Hyde Park', 'N.Y.', ' Marsh&McLennan Companies, Inc.,'),
 (' New York City', 'N.Y.', ' Fred Alger Management, Inc.,'),
 (' Bronx', 'N.Y.', ' Windows on the World,'),
 ('', '', ' Cantor Fitzgerald,'),
 (' Manalapan'

## Progress so far

In [111]:
# Imports
from composable import pipeable
from composable.strict import map

In [112]:
# Reg Ex for a line
line_parts = re.compile(r'''^(.+),
(
      \s\?\?                          # ??
    | \s\d{1,3}                       # or age
),
(.*?)                                 # Includes hometown and 
(
        \sPassenger,                  # Optional flight status
    |   \sFlightsCrew,
)?
(
      \sUnited\s\d{2,3},              # Optional flight
    | \sAmericans\d{2,3},
)?
(
       \sWorld\sTrade\sCenter         # Location
    |  \sPentagon
    |  \sShanksville,\sPa
)
(
    ,\sdied\s\d{1,2}/\d{1,2}/\d{1,2}  # Optional date of death
)?
\.$''', re.VERBOSE)

In [500]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))
get_line_parts = pipeable(lambda line: line_parts.search(line).groups(default=''))
remove_commas = lambda s: s.replace(',', '')
# New
fix_nyc = pipeable(lambda line: line.replace(', New York City,', ', New York City, N.Y.,'))

In [501]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

In [502]:
split_lines =  (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(fix_nyc)
                >> map(get_line_parts)
                )
split_lines

[('Gordon M. Aamoth, Jr.',
  ' 32',
  " Sandler O'Neill + Partners,",
  '',
  '',
  ' World Trade Center',
  ''),
 ('Edelmiro Abad',
  ' 54',
  ' Brooklyn, N.Y., Fiduciary Trust Company International,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Marie Rose Abad',
  ' 49',
  ' Keefe, Bruyette&Woods, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Andrew Anthony Abate',
  ' 37',
  ' Melville, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Vincent Paul Abate',
  ' 40',
  ' Brooklyn, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Laurence Christopher Abel',
  ' 37',
  ' New York City, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Alona Abraham',
  ' 30',
  ' Ashdod, Israel,',
  ' Passenger,',
  ' United 175,',
  ' World Trade Center',
  ''),
 ('William F. Abrahamson',
  ' 55',
  ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Richard Anth

In [506]:
names =  (split_lines
        >> map(get(0))
        >> map(remove_commas)
        )
names

['Gordon M. Aamoth Jr.',
 'Edelmiro Abad',
 'Marie Rose Abad',
 'Andrew Anthony Abate',
 'Vincent Paul Abate',
 'Laurence Christopher Abel',
 'Alona Abraham',
 'William F. Abrahamson',
 'Richard Anthony Aceto',
 'Heinrich Bernhard Ackermann',
 'Paul Acquaviva',
 'Christian Adams',
 'Donald LaRoy Adams',
 'Patrick Adams',
 'Shannon Lewis Adams',
 'Stephen George Adams',
 'Ignatius Udo Adanga',
 'Christy A. Addamo',
 'Terence Edward Adderley Jr.',
 'Sophia B. Addo',
 'Lee Adler',
 'Daniel Thomas Afflitto',
 'Emmanuel Akwasi Afuakwah',
 'Alok Agarwal',
 'Mukul Kumar Agarwala',
 'Joseph Agnello',
 'David Scott Agnes',
 'Joao Alberto da Fonseca Aguiar Jr.',
 'Brian G. Ahearn',
 'Jeremiah Joseph Ahern',
 'Joanne Marie Ahladiotis',
 'Shabbir Ahmed',
 'Terrance Andre Aiken',
 'Godwin O. Ajala',
 'Trudi M. Alagero',
 'Andrew Alameno',
 'Margaret Ann Alario',
 'Gary M. Albero',
 'Jon Leslie Albert',
 'Peter Craig Alderman',
 'Jacquelyn Delaine Aldridge-Frederick',
 'David D. Alger',
 'Ernest Ali

In [507]:
troubling_bit = (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(fix_nyc)
                >> map(get_line_parts)
                >> map(get(2))
                )
troubling_bit

[" Sandler O'Neill + Partners,",
 ' Brooklyn, N.Y., Fiduciary Trust Company International,',
 ' Keefe, Bruyette&Woods, Inc.,',
 ' Melville, N.Y., Cantor Fitzgerald,',
 ' Brooklyn, N.Y., Cantor Fitzgerald,',
 ' New York City, N.Y., Cantor Fitzgerald,',
 ' Ashdod, Israel,',
 ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
 ' Marsh&McLennan Companies, Inc.,',
 ' Aon Corporation,',
 ' Glen Rock, N.J., Cantor Fitzgerald,',
 '',
 ' Cantor Fitzgerald,',
 ' Fuji Bank, Ltd. security,',
 ' Cantor Fitzgerald,',
 ' New York City, N.Y., Windows on the World,',
 ' Bronx, N.Y., New York Metropolitan Transportation Council,',
 ' New Hyde Park, N.Y., Marsh&McLennan Companies, Inc.,',
 ' New York City, N.Y., Fred Alger Management, Inc.,',
 ' Bronx, N.Y., Windows on the World,',
 ' Cantor Fitzgerald,',
 ' Manalapan, N.J., Cantor Fitzgerald,',
 ' Windows on the World,',
 ' Cantor Fitzgerald,',
 ' Fiduciary Trust Company International,',
 ' Belle Harbor, N.Y., New York City Fire Department,',

In [537]:
state = re.compile('''
^(.*?)
,?\s                    # Optional comman
(
       N\.Y\.           
    |  N\.J\.
    |  D\.C\.
    |  N\.H\.
    |  N\.M\.
    |  N\.C\.
    |  R.I.
    |  Md\.
    |  Pa\.
    |  Va\.
    |  Ga\.
    |  La\.
    |  Mass\.
    |  Calif\.
    |  Ariz\.
    |  Fla\.
    |  Ill\.
    |  Conn\.
    |  Hawaii
    |  Iowa
    |  Maine
    |  New\sHampshire
    |  New\sJersey
    |  New\sYork
    |  Ohio
    |  Pennsylvania
    |  Texas
    |  Utah
    |  Virginia
    |  Japan
    |  India
    |  Germany
    |  Manitoba,\sCanada
    |  New\sSouth\sWales,\sAustralia
    |  England,\sUnited\sKingdom
)
,
(.*?)$
''', re.VERBOSE)

In [538]:
( troubling_bit
 >> map(split_troubling_bit)
)

[('', '', " Sandler O'Neill + Partners,"),
 (' Brooklyn', 'N.Y.', ' Fiduciary Trust Company International,'),
 ('', '', ' Keefe, Bruyette&Woods, Inc.,'),
 (' Melville', 'N.Y.', ' Cantor Fitzgerald,'),
 (' Brooklyn', 'N.Y.', ' Cantor Fitzgerald,'),
 (' New York City', 'N.Y.', ' Cantor Fitzgerald,'),
 ('', '', ' Ashdod, Israel,'),
 (' Westchester County', 'N.Y.', ' Marsh&McLennan Companies, Inc.,'),
 ('', '', ' Marsh&McLennan Companies, Inc.,'),
 ('', '', ' Aon Corporation,'),
 (' Glen Rock', 'N.J.', ' Cantor Fitzgerald,'),
 ('', '', ''),
 ('', '', ' Cantor Fitzgerald,'),
 ('', '', ' Fuji Bank, Ltd. security,'),
 ('', '', ' Cantor Fitzgerald,'),
 (' New York City', 'N.Y.', ' Windows on the World,'),
 (' Bronx', 'N.Y.', ' New York Metropolitan Transportation Council,'),
 (' New Hyde Park', 'N.Y.', ' Marsh&McLennan Companies, Inc.,'),
 (' New York City', 'N.Y.', ' Fred Alger Management, Inc.,'),
 (' Bronx', 'N.Y.', ' Windows on the World,'),
 ('', '', ' Cantor Fitzgerald,'),
 (' Manalapan'

<h2> <font color="red"> Exercise 4.5.4 </font> </h2>

Clean up each part of the troubling bits, then comma join this section into 1 string.

**Hint:** Be sure to remove any problematic commas.

In [132]:
remove_period = lambda s: s.replace('.','')
( troubling_bit
    >> map(split_troubling_bit)
    >> map(get(1))
    >> map(remove_period)
    >> map(remove_commas)
)

['',
 'NY',
 '',
 'NY',
 'NY',
 'NY',
 '',
 'NY',
 '',
 '',
 'NJ',
 '',
 '',
 '',
 '',
 'NY',
 'NY',
 'NY',
 'NY',
 'NY',
 '',
 'NJ',
 '',
 '',
 '',
 'NY',
 'NY',
 'NJ',
 'NY',
 '',
 'NY',
 'NY',
 '',
 '',
 'NY',
 'NJ',
 'NY',
 'NJ',
 'NY',
 'NY',
 'NY',
 'NY',
 'NY',
 '',
 'NY',
 '',
 'NY',
 'NY',
 'NJ',
 'Mass',
 'NY',
 '',
 '',
 'NY',
 '',
 '',
 '',
 '',
 '',
 '',
 'DC',
 '',
 '',
 'NY',
 'NY',
 'NY',
 'NJ',
 'NY',
 'NY',
 '',
 'NJ',
 '',
 'NY',
 'NY',
 '',
 '',
 'NY',
 'NY',
 'NJ',
 '',
 'NJ',
 'NY',
 'NY',
 'NY',
 '',
 '',
 'NY',
 '',
 'NY',
 'NY',
 'NJ',
 'NY',
 'NJ',
 'Mass',
 '',
 'NY',
 'NJ',
 'NY',
 'NY',
 'NY',
 'NY',
 'NY',
 'NY',
 '',
 'NJ',
 'NY',
 '',
 '',
 'NJ',
 'NY',
 'NY',
 'NY',
 'NY',
 'NY',
 'Mass',
 '',
 '',
 'NY',
 '',
 '',
 'NY',
 'NY',
 'NY',
 'NJ',
 'NY',
 'NY',
 'Mass',
 '',
 '',
 'NY',
 '',
 '',
 'NJ',
 '',
 'NY',
 'NY',
 'NJ',
 'NY',
 'NY',
 'NY',
 'NY',
 'NJ',
 '',
 'NY',
 'NY',
 'NJ',
 '',
 'NJ',
 'Mass',
 '',
 'NJ',
 '',
 'NJ',
 'NJ',
 '',
 '',
 'NY',
 

## Combining the parts back together.

We can combine the parts back together using the `zip` function.

In [136]:
from composable.strict import zipOnto
from composable.list import to_list
(zip(names, ages,fixed_troubling_bits)
 >> to_list
 >> map(comma_join)
)

TypeError: zip argument #3 must support iteration

<h2> <font color="red"> Exercise 4.5.4 </font> </h2>

Use `zip` to combine all part of the data and write the result out to a file called `911_Deaths_Fixed.csv` 

In [129]:
from composable.strict import zipOnto
from composable.list import to_list
(zip(names, ages, fixed_troubling_bits, locations, flight, passenger, death)
 >> to_list
 >> map(comma_join)
)

NameError: name 'fixed_troubling_bits' is not defined